In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout

In [2]:
from sklearn.metrics import classification_report 
from sklearn.metrics import f1_score ,accuracy_score

In [3]:
import cv2, imutils
def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
    [0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()

In [4]:
import os
import pandas as pd
cwd = os.getcwd()
print(cwd)
df = pd.read_csv(cwd + "\\..\\data_csv\\all_data.csv")
print(df)

c:\Users\Pongsakorn\OneDrive\Desktop\Working\FacialLandMarks\facial_cleandata\neural_network
                                 image   emotion
0      facial-expressions_2868588k.jpg     anger
1      facial-expressions_2868585k.jpg  surprise
2      facial-expressions_2868584k.jpg   disgust
3      facial-expressions_2868582k.jpg      fear
4               Aaron_Eckhart_0001.jpg   neutral
...                                ...       ...
34670      SURPRISE/surprise (988).jpg  SURPRISE
34671       SURPRISE/surprise (99).jpg  SURPRISE
34672      SURPRISE/surprise (993).jpg  SURPRISE
34673      SURPRISE/surprise (994).png  SURPRISE
34674      SURPRISE/surprise (996).jpg  SURPRISE

[34675 rows x 2 columns]


In [5]:
raw_images =  []
labels = []
for i, row in df.iterrows():
    image_path = row['image']
    label = row['emotion']
    full_image_path = os.path.dirname(cwd) + "\\cleaned_images\\" + image_path
    image = cv2.imread(full_image_path)
    pixels = extract_color_histogram(image)

    raw_images.append(pixels)
    labels.append(label)
    if i > 0 and i % 1000 == 0: print('[INFO] processed {}/{}'.format(i, len(df)))

[INFO] processed 1000/34675
[INFO] processed 2000/34675
[INFO] processed 3000/34675
[INFO] processed 4000/34675
[INFO] processed 5000/34675
[INFO] processed 6000/34675
[INFO] processed 7000/34675
[INFO] processed 8000/34675
[INFO] processed 9000/34675
[INFO] processed 10000/34675
[INFO] processed 11000/34675
[INFO] processed 12000/34675
[INFO] processed 13000/34675
[INFO] processed 14000/34675
[INFO] processed 15000/34675
[INFO] processed 16000/34675
[INFO] processed 17000/34675
[INFO] processed 18000/34675
[INFO] processed 19000/34675
[INFO] processed 20000/34675
[INFO] processed 21000/34675
[INFO] processed 22000/34675
[INFO] processed 23000/34675
[INFO] processed 24000/34675
[INFO] processed 25000/34675
[INFO] processed 26000/34675
[INFO] processed 27000/34675
[INFO] processed 28000/34675
[INFO] processed 29000/34675
[INFO] processed 30000/34675
[INFO] processed 31000/34675
[INFO] processed 32000/34675
[INFO] processed 33000/34675
[INFO] processed 34000/34675


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(np.array(raw_images), np.array(labels), random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [8]:
import tensorflow as tf
def classification_model():
    model = Sequential()
    model.add(Dense(124, activation='relu', input_shape=(X_train.shape[1],)))
    # model.add(Dropout(0.2))
    model.add(Dense(124, activation='relu'))
    # model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [30]:
model = classification_model()
model.fit(X_train, y_train, validation_data=(X_val, y_val,), epochs=100, verbose=2)

Epoch 1/100


C:\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


610/610 - 2s - loss: 2.5518 - accuracy: 0.1338 - val_loss: 2.3365 - val_accuracy: 0.1195 - 2s/epoch - 3ms/step
Epoch 2/100
610/610 - 1s - loss: 2.3193 - accuracy: 0.1549 - val_loss: 2.2985 - val_accuracy: 0.1769 - 1s/epoch - 2ms/step
Epoch 3/100
610/610 - 1s - loss: 2.2905 - accuracy: 0.1748 - val_loss: 2.2610 - val_accuracy: 0.1949 - 1s/epoch - 2ms/step
Epoch 4/100
610/610 - 1s - loss: 2.2603 - accuracy: 0.1837 - val_loss: 2.2362 - val_accuracy: 0.2024 - 1s/epoch - 2ms/step
Epoch 5/100
610/610 - 1s - loss: 2.2405 - accuracy: 0.1921 - val_loss: 2.2219 - val_accuracy: 0.1982 - 1s/epoch - 2ms/step
Epoch 6/100
610/610 - 1s - loss: 2.2313 - accuracy: 0.1893 - val_loss: 2.2146 - val_accuracy: 0.2032 - 1s/epoch - 2ms/step
Epoch 7/100
610/610 - 1s - loss: 2.2212 - accuracy: 0.1971 - val_loss: 2.2096 - val_accuracy: 0.2058 - 1s/epoch - 2ms/step
Epoch 8/100
610/610 - 1s - loss: 2.2178 - accuracy: 0.1942 - val_loss: 2.2082 - val_accuracy: 0.2044 - 1s/epoch - 2ms/step
Epoch 9/100
610/610 - 1s - l

In [10]:
value = model.predict(X_test)
y_pred =np.argmax(value,axis=1)
y_true = np.argmax(y_test,axis=1)

In [11]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       579
           1       0.00      0.00      0.00       536
           2       0.00      0.00      0.00       636
           3       0.12      0.01      0.02       641
           4       0.18      0.05      0.08      1120
           5       0.17      0.50      0.25      1164
           6       0.00      0.00      0.00       741
           7       0.23      0.12      0.16       803
           8       0.00      0.00      0.00        46
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00      1066
          13       0.22      0.80      0.35      1249
          14       0.00      0.00      0.00        17
          15       0.00      0.00      0.00        62

    accuracy                           0.20      8669
   macro avg       0.06   

C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
value = model.predict(X_test)
y_pred = np.argmax(value,axis=1)
y_true = np.argmax(y_test,axis=1)

In [29]:
y_pred_arr = list(y_pred)
print("len : ", len(y_pred_arr))
score = 0
for i in range(len(y_pred_arr)):
    if(list(y_pred)[i] == list(y_true)[i]): score += 1
max_score = len(list(y_pred))
print(f"Final Score : {score}/{max_score}")
print("Accuracy : ", 100 * score / max_score)


len :  8669
Final Score : 1739/8669
Accuracy :  20.059983850501787


In [ ]:
# Hyperparameter Process #

In [12]:
from scipy.integrate import odeint
# Parameters and time for FOPDT model
ns = 10000
t = np.linspace(0,ns-1,ns)
u = np.zeros(ns)
# Additional FOPDT parameters
yp0 = 0.0
u0 = u[0]
Km = 0.67
taum = 160.0
def fopdt(y,t,um,Km,taum):
    # arguments
    #  y      = output
    #  t      = time
    #  uf     = input linear function (for time shift)
    #  Km     = model gain
    #  taum   = model time constant
    # calculate derivative
    dydt = (-(y-yp0) + Km * (um-u0))/taum
    return dydt

def sim_model(Km,taum):
    # array for model values
    ym = np.zeros(ns)
    # initial condition
    ym[0] = yp0
    # loop through time steps    
    for i in range(0,ns-1):
        ts = [t[i],t[i+1]]
        y1 = odeint(fopdt,ym[i],ts,args=(u[i],Km,taum))
        ym[i+1] = y1[-1]
    return ym

In [ ]:
import random
end = 60 # leave 1st minute of u as 0
while end <= ns:
    start = end
    end += random.randint(300,900) # keep new Q1s value for anywhere from 5 to 15 minutes
    u[start:end] = random.randint(0,100)

In [13]:
# Simulate FOPDT model
y = sim_model(Km,taum)

In [14]:
# Add Gaussian noise
noise = np.random.normal(0,0.2,ns)
y += noise

In [16]:
from sklearn.preprocessing import MinMaxScaler
# Scale data
data = np.vstack((u,y)).T
s = MinMaxScaler(feature_range=(0,1))
data_s = s.fit_transform(data)